In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import zcode.math as zmath

# `regress()`

In [ ]:
def test_regress_1d(num, noise=0.2, rtol=0.2, truth=None, plot=False, seed=16284):
    np.random.seed(seed)
    if truth is None:
        # truth = [2.2, -4.7]
        truth = np.random.uniform(-10, 10, 2)
        
    xx = np.linspace(0.0, 1.0, num)
    yy = truth[0] * xx + truth[1]
    noise *= (yy.max() - yy.min())
    print(f"{noise=}")
    yy += np.random.normal(0.0, noise, yy.size)

    coeff, zz = zmath.numeric.regress(xx, yy)

    if plot:
        plt.scatter(xx, yy)
        plt.plot(xx, zz, 'k--')

    err = (truth - coeff) / truth
    err = np.max(err)
    print(f"{truth=}, {coeff=}")
    print(f"{err=:.4e}")
    if not np.isclose(truth[0], coeff[0], rtol=rtol):
        raise ValueError

    if not np.isclose(truth[1], coeff[1], atol=noise):
        raise ValueError        
        
    return


def test_regress_2d(npars, noise=0.2, rtol=0.2, truth=None, plot=False):
    size = 20
    np.random.seed(16284)
    if truth is None:
        # truth = [2.2, -4.7]
        truth = np.random.uniform(-10, 10, (2, npars))
    else:
        truth = np.array([truth for ii in range(npars)]).T        
        
    # xx = np.random.uniform(0.0, 1.0, (size, npars))
    # xx.sort(axis=0)
    xx = np.array([np.linspace(0.0, 1.0, size) for ii in range(npars)]).T
    
    yy = truth[0, np.newaxis, :] * xx + truth[1, np.newaxis, :]
    print(f"{yy.shape=}")
    noise = noise * (yy.max(axis=0) - yy.min(axis=0))
    print(f"{noise.shape=}")
    yy += np.random.normal(0.0, noise, (size, npars))
    print(f"{xx.shape=}, {yy.shape=}")
    coeff, zz = zmath.numeric.regress(xx, yy)
    print(f"{coeff.shape=}, {zz.shape=}")
    
    if plot:
        for ii in range(npars):
            hh, = plt.plot(xx[:, ii], zz[:, ii], ls='--', alpha=0.5)
            plt.scatter(xx[:, ii], yy[:, ii], color=hh.get_color(), alpha=0.5)

    err = (truth - coeff) / truth
    err = np.max(err)
    print(f"{truth=}, {coeff=}")
    print(f"{err=:.4e}")
    if not np.allclose(truth[0], coeff[0], rtol=rtol):
        raise ValueError

    if not np.allclose(truth[1], coeff[1], atol=noise):
        raise ValueError        
        
    return


'''
def regress(xx, yy):
    """Solve  `ax + b = y`
    """
    aa = np.concatenate([xx[np.newaxis, :], np.ones_like(xx)[np.newaxis, :]])
    ata = np.einsum('ji...,ki...->jk...', aa, aa)
    aty = np.einsum('ji...,i...->j...', aa, yy)
    if np.ndim(ata) > 2:
        ata = np.moveaxis(ata, -1, 0)

    ata_inv = np.linalg.inv(ata)
    if np.ndim(ata) > 2:
        ata_inv = np.moveaxis(ata_inv, 0, -1)

    # sol = ata_inv * aty
    coeff = np.einsum('ij...,j...->i...', ata_inv, aty)
    zz = np.einsum('ji...,j...->i...', aa, coeff)
    return coeff, zz
'''


def test_regress_3d(shape, noise=0.2, rtol=0.2, truth=None, plot=False):
    size = 20
    np.random.seed(16284)
    shape = tuple(shape)
    # Slice that will broadcast from (N,) to (N,shape)
    broad = [slice(None),] + [None for ii in range(len(shape))]
    broad = tuple(broad)
    if truth is None:
        truth = np.random.uniform(-10, 10, (2,) + shape)
    else:
        truth = np.ones((2,) + tuple(shape)) * np.array(truth)[broad]
        
    xx = np.ones((size,) + tuple(shape)) * np.linspace(0.0, 1.0, size)[broad]
    print(f"{xx.shape=}, {truth.shape=}")
    
    # Slice that will broadcast from (N,) to (shape,N,)
    # broad = [None for ii in range(len(shape))] + [slice(None),]
    yy = truth[0, np.newaxis, ...] * xx + truth[1, np.newaxis, ...]
    print(f"{yy.shape=}")
    noise = noise * (yy.max(axis=0) - yy.min(axis=0))
    print(f"{noise.shape=}")
    yy += np.random.normal(0.0, noise, (size,) + shape)
    print(f"{xx.shape=}, {yy.shape=}")
    coeff, zz = zmath.numeric.regress(xx, yy)
    print(f"{coeff.shape=}, {zz.shape=}")
    
    if plot:
        xx = np.moveaxis(xx, 0, -1)
        yy = np.moveaxis(yy, 0, -1)
        zz = np.moveaxis(zz, 0, -1)
        for ii in np.ndindex(shape):
            print(f"{ii=}")
            hh, = plt.plot(xx[ii], zz[ii], ls='--', alpha=0.5)
            plt.scatter(xx[ii], yy[ii], color=hh.get_color(), alpha=0.5)

    err = (truth - coeff) / truth
    err = np.max(err)
    print(f"{truth=}, {coeff=}")
    print(f"{err=:.4e}")
    if not np.allclose(truth[0], coeff[0], rtol=rtol):
        raise ValueError

    if not np.allclose(truth[1], coeff[1], atol=noise):
        raise ValueError        
        
    return

In [ ]:
# With zero noise the solution should be perfect
for ii in range(10):
    test_regress_1d(20, noise=0.0, rtol=1e-6, seed=None)

In [ ]:
truth = None
plot = True
test_regress_1d(20, truth=truth, plot=plot)
print('\n')
plt.show()
test_regress_2d(3, truth=truth, plot=plot, rtol=0.5)
print('\n')
plt.show()
test_regress_3d((2, 3), truth=truth, plot=plot, rtol=0.5)
plt.show()

In [ ]:
# Make sure ND solutions are consistent with 1D
# ---------------------------------------------

def test_consistent(shp):
    xx = np.random.uniform(-10, 10, shp)
    truth = np.random.uniform(-10, 10, (2,) + shp[1:])
    yy = xx * truth[0, np.newaxis] + truth[1, np.newaxis]

    cc, zz = zmath.numeric.regress(xx, yy)

    xx = np.moveaxis(xx, 0, -1)
    yy = np.moveaxis(yy, 0, -1)
    cc = np.moveaxis(cc, 0, -1)
    zz = np.moveaxis(zz, 0, -1)
    for idx in np.ndindex(shp[1:]):    
        err = f"{idx=} "
        _cc, _zz = zmath.numeric.regress(xx[idx], yy[idx])
        if not np.allclose(_cc, cc[idx]):
            err += f"coefficients do not match!  1D: {_cc}, ND: {cc[idx]}"
            raise ValueError(err)

        if not np.allclose(_zz, zz[idx]):
            err += f"model solutions do not match!  1D: {_zz}, ND: {zz[idx]}"
            raise ValueError(err)

    return

test_consistent((20, 3, 4, 5))
test_consistent((20, 4, 7))